<a href="https://colab.research.google.com/github/darwinagudeloh/FDL_2024-2_UDEA/blob/main/03_Preprocesamiento_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar librerías
import requests
from io import StringIO
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# Cargar el archivo CSV desde GitHub
file_path = "https://raw.githubusercontent.com/darwinagudeloh/FDL_2024-2_UDEA/refs/heads/main/combined_wind_data.csv"

# Descargar el archivo desde GitHub
response = requests.get(file_path)
if response.status_code == 200:
    # Leer el contenido del archivo en un DataFrame
    data = pd.read_csv(StringIO(response.text))
data = pd.read_csv(file_path)

In [3]:
# Convertir la columna 'Date' a tipo datetime y establecer como índice
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [4]:
# Normalización de las Variables
# Seleccionar columnas numéricas para normalización (excluir 'Date')
columns_to_normalize = [col for col in data.columns if 'WS10M_MAX' in col]
scaler = MinMaxScaler(feature_range=(0, 1))
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

In [5]:
# Separar datos en características (X) y objetivo (y)
X_scaled = data.drop(columns=['WS10M_MAX_punto6'])
y_scaled = data['WS10M_MAX_punto6']

In [6]:
# Función para crear secuencias con ventanas y horizonte de pronóstico
def create_sequences(data, target, window_size, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - window_size - forecast_horizon + 1):
        X.append(data[i:i + window_size])  # Ventana de entrada
        y.append(target[i + window_size:i + window_size + forecast_horizon])  # Ventana de pronóstico
    return np.array(X), np.array(y)

In [7]:
# Parámetros de ventana y horizonte
window_size = 30  # Días de ventana
forecast_horizon = 7  # Días a pronosticar

In [8]:
# Crear secuencias
X, y = create_sequences(X_scaled, y_scaled, window_size, forecast_horizon)

In [9]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)